# Retrieval and Search
In this notebook, we will use a search engine program that will help us:
- Indexing documents
- Performing searh

In [69]:
# Run this script to download the minsearch.py file
# Optionally, you can use the wget command to download the file
# !curl https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py -o minsearch.py

In [70]:
# Run this script to download the documents.json file containing the questions and answers
# of the FAQ documents
# !curl https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json -o documents.json

In [4]:
import minsearch
import json

In [5]:
# Open json file containing the questions and answers of the FAQ documents
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [6]:
# Transform to capture the documents, and create a new 'course' field
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
documents[:2]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'}]

In [8]:
# Index the documents using the minsearch.Index class
# The text_fields parameter specifies the fields that will be used for full-text search
# The keyword_fields parameter specifies the fields that will be used for keyword search
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [9]:
# Fits the index with the provided documents.
index.fit(documents)

In [77]:
# The search method returns a list of documents that match the query
# The parameters are:
# query: the search query
# num_results: the number of results to return
# boost_dict: a dictionary that specifies the weight of each field in the search query
# filter_dict: a dictionary that specifies the filter to apply to the search results

q = "the course has already started, can I still enroll?"
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [78]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

# Generating Answers with OpenAI GPT
Using the documents retrieved using our search engine, we will now feed the context of our LLM for the question asked. 
- Invoke OpenAI API
- Build a prompt
- Get an answer

In [20]:
from openai import OpenAI
from dotenv import load_dotenv
import os

In [80]:
# This is our query
q

'the course has already started, can I still enroll?'

In [81]:
# Here we build the context we are going to use to generate the answer
# We format the documents information in a more readable way
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [82]:
# We declare the template we are going to use to generate the prompt
prompt_template = """
You're a course teaching assistant. A student asks you the following question.
Use only the facts from the CONTEXT when answering the QUESTION.
If the answer is not in the context, output NONE.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [83]:
# We put everything together to generate the prompt
prompt = prompt_template.format(question=q, context=context).strip()

In [89]:
print(prompt)

You're a course teaching assistant. A student asks you the following question.
Use only the facts from the CONTEXT when answering the QUESTION.
If the answer is not in the context, output NONE.

QUESTION: the course has already started, can I still enroll?

CONTEXT: 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

sec

In [21]:
# We load our environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [22]:
# We create an OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

In [87]:
# Finally, we generate the answer using the OpenAI API
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": prompt},
    ]
)

response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."